# Advent of Code challenge 2021

## Day 4: Giant Squid

### Part 1 - Play Bingo and Find Winning Board Grid

Good references I found helpful:
- numpy.all() - https://pythonexamples.org/numpy-all/
- numpy.all() - https://coderzcolumn.com/tutorials/python/dask-array-guide-to-work-with-large-arrays-in-parallel

In [3]:
import dask.array as da
import numpy as np

In [4]:
test_input = """
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
"""

### Create a class to solve this problem by looping over bingo numbers in succession until a winning board is found

In [20]:
class Bingo:
    def __init__(self, puz_input):
        self.puz_input = puz_input
        self.bingo_nums = self.get_bingo_nums()
        self.bingo_grids = self.make_bingo_grids()
        self.bingo_trackers = self.make_bingo_trackers()
         
    
    def __repr__(self):
        info = [f"\nBoard score: {self.score_win_board}",
                f"Bingo number: {self.win_bingo_num}",
                f"Final score: {self.score_win_board * self.win_bingo_num}"]
        return '\n'.join(info)
    
            
    def get_bingo_nums(self):
        """Extract first line of puzzle input as bingo numbers to be 'called'"""
        bingo_num_list = self.puz_input.strip().split('\n')[0]
        return [int(bingo) for bingo in bingo_num_list.split(',')]
    
    
    def make_bingo_grids(self):
        """Parse puzzle input to create arrays of 5x5 bingo grids, 1 grid per chunk."""
        bingo_lines = self.puz_input.strip().split('\n')[2:]

        bingo_grids = []
        grid_list = []

        for line in bingo_lines:
            if line == '':
                bingo_grids.append(np.array(grid_list))
                grid_list = []
            else:
                single_space_line = line.strip().replace('  ', ' ')
                grid_list.append([int(val) for val in single_space_line.split(' ')])
        else:
            bingo_grids.append(np.array(grid_list))

        return da.from_array(bingo_grids, chunks = (1, 5, 5))
    
    
    def make_bingo_trackers(self):
        """Make bingo tracker grids that track when a bingo grid has a number called."""
        np_arr = [np.array([[False] * 5] * 5)] * len(self.bingo_grids)
        return da.from_array(np_arr, chunks = (1, 5, 5))
    
    
    def get_winning_board(self):
        """Find winning bingo board by summing along each axis - row, col - until one is all True."""
        if da.all(self.bingo_trackers, axis = 1).any():
            return da.all(self.bingo_trackers, axis = 1).any(axis = 1)

        if da.all(self.bingo_trackers, axis = 2).any():
            return da.all(self.bingo_trackers, axis = 2).any(axis = 1)

        return da.array([False, False])
    
    
    def call_bingo_nums(self):
        """Call each bingo number and update bingo trackers for each board 
            until a winning board is found.
        """
        for i, bingo_num in enumerate(self.bingo_nums):
            print(i, end=', ')
            self.bingo_trackers[da.isin(self.bingo_grids, bingo_num)] = True
            winning_board = self.get_winning_board()
            if winning_board.any():
                print("Found winning board.")
                self.bingo_grids[self.bingo_trackers] = 0
                return (bingo_num, self.bingo_grids[winning_board].compute())
        else:
            return (bingo_num, self.bingo_trackers.compute())
        
        
    def solve(self):
        """Main function to solve bingo board puzzle."""
        self.win_bingo_num, self.winning_board = self.call_bingo_nums()
        self.score_win_board = da.sum(self.winning_board, axis=1).sum().compute()

In [21]:
# Test input with given example
bingo_test = Bingo(puz_input=test_input)
bingo_test.solve()
print(bingo_test)

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, Found winning board.

Board score: 188
Bingo number: 24
Final score: 4512


In [22]:
# Run with final puzzle input
with open('adv_2021_d4_input.txt', 'r') as f:
    bingo_pt1_final = Bingo(puz_input=f.read())
    bingo_pt1_final.solve()
    print(bingo_pt1_final)

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, Found winning board.

Board score: 439
Bingo number: 57
Final score: 25023


### Part 2 - Finding Last Winning Board

It might be wise to try a different strategy: let the squid win.
Find out which board will win *last*.

In [553]:
# Identify last non-winning board
da.all(bingo_trackers, axis = 2).any(axis = 1).compute()

array([False, False,  True])

In [537]:
num_bingo_grids = len(bingo_grids)
da.all(bingo_trackers, axis = 1).any(axis = 1).sum().compute()

0

In [64]:
class Bingo2:
    def __init__(self, puz_input):
        self.puz_input = puz_input
        self.bingo_nums = self.get_bingo_nums()
        self.bingo_grids = self.make_bingo_grids()
        self.num_bingo_grids = len(self.bingo_grids)
        self.bingo_trackers = self.make_bingo_trackers()
         
    
    def __repr__(self):
        info = [f"\nBoard score: {self.score_win_board}",
                f"Bingo number: {self.win_bingo_num}",
                f"Final score: {self.score_win_board * self.win_bingo_num}"]
        return '\n'.join(info)
    
            
    def get_bingo_nums(self):
        """Extract first line of puzzle input as bingo numbers to be 'called'"""
        bingo_num_list = self.puz_input.strip().split('\n')[0]
        return [int(bingo) for bingo in bingo_num_list.split(',')]
    
    
    def make_bingo_grids(self):
        """Parse puzzle input to create arrays of 5x5 bingo grids, 1 grid per chunk."""
        bingo_lines = self.puz_input.strip().split('\n')[2:]

        bingo_grids = []
        grid_list = []

        for line in bingo_lines:
            if line == '':
                bingo_grids.append(np.array(grid_list))
                grid_list = []
            else:
                single_space_line = line.strip().replace('  ', ' ')
                grid_list.append([int(val) for val in single_space_line.split(' ')])
        else:
            bingo_grids.append(np.array(grid_list))

        return da.from_array(bingo_grids, chunks = (1, 5, 5))
    
    
    def make_bingo_trackers(self):
        """Make bingo tracker grids that track when a bingo grid has a number called."""
        np_arr = [np.array([[False] * 5] * 5)] * self.num_bingo_grids
        return da.from_array(np_arr, chunks = (1, 5, 5))
    
    
    def get_winning_board(self):
        """Find winning bingo board by summing along each axis - row, col - until one is all True."""
        if da.all(self.last_bingo_tracker, axis = 0).any():
            return da.all(self.last_bingo_tracker, axis = 0).any(axis = 0)

        if da.all(self.last_bingo_tracker, axis = 1).any():
            return da.all(self.last_bingo_tracker, axis = 1).any(axis = 0)

        return da.array([False, False])
    
    
    def get_last_winning_board(self):
        """Sum along each bingo tracker grid axis - row, col - and return last winning board index."""
        col_sum = da.all(self.bingo_trackers, axis = 1).any(axis = 1).sum()
        row_sum = da.all(self.bingo_trackers, axis = 2).any(axis = 1).sum()

        if max(col_sum, row_sum) == (self.num_bingo_grids - 1):
            winning_board_tracker = da.all(self.bingo_trackers, axis = 2).any(axis = 1).compute()
            last_win_board_idx = np.where(winning_board_tracker == False)[0][0]
            return (self.bingo_grids[last_win_board_idx].compute(), 
                    self.bingo_trackers[last_win_board_idx].compute())
        else:
            return (da.array([False, False]), da.array([False, False]))
    
    
    def call_bingo_nums(self):
        """Call each bingo number and update bingo trackers for each board until all 
            but 1 winning board is found.
        """
        for bingo_num in self.bingo_nums:
            print(bingo_num, end=', ')
            self.bingo_trackers[da.isin(self.bingo_grids, bingo_num)] = True
            self.winning_board, self.last_bingo_tracker = self.get_last_winning_board()

            if self.winning_board.any():
                print("Found last non-winning board.")
                break
        else:
            return bingo_num, self.winning_board

        # Check if last board has won yet
        self.last_winning_board = self.get_winning_board()
        if self.last_winning_board.any():
            print("Found last winning board.")
            self.winning_board[self.last_bingo_tracker] = 0
            return bingo_num, self.winning_board
        
        # Found last board to win, but it hasn't won yet. Keep calling numbers until it wins.
        for bingo_num in self.bingo_nums[i + 1:]:            
            self.last_bingo_tracker[np.isin(self.winning_board, bingo_num)] = True
            self.last_winning_board = self.get_winning_board()
            
            if self.last_winning_board.any():
                print("Found last winning board.")
                self.winning_board[self.last_bingo_tracker] = 0
                return bingo_num, self.winning_board
        
        
    def solve(self):
        """Main function to solve bingo board puzzle."""
        self.win_bingo_num, self.winning_board = self.call_bingo_nums()
        self.score_win_board = da.sum(self.winning_board, axis=1).sum().compute()

In [55]:
# Test input with given example
bingo_test = Bingo2(puz_input=test_input)
bingo_test.solve()
print(bingo_test)

7, 4, 9, 5, 11, 17, 23, 2, 0, 14, 21, 24, 10, 16, Found last non-winning board.
13, Found last winning board.

Board score: 148
Bingo number: 13
Final score: 1924


In [65]:
# Run with final puzzle input
with open('adv_2021_d4_input.txt', 'r') as f:
    bingo_pt2_final = Bingo2(puz_input=f.read())
    bingo_pt2_final.solve()
    print(bingo_pt2_final)

18, 99, 39, 89, 0, 40, 52, 72, 61, 77, 69, 51, 30, 83, 20, 65, 93, 88, 29, 22, 14, 82, 53, 41, 76, 79, 46, 78, 56, 57, 24, 36, 38, 11, 50, 1, 19, 26, 70, 4, 54, 3, 84, 33, 15, 21, 9, 58, 64, 85, 10, 66, 17, 43, 31, 27, 2, 5, 95, 96, 16, 97, 12, 34, 74, 67, 86, 23, 49, 8, 59, 45, 68, 91, 25, 48, 13, 28, 81, 94, 92, 42, 7, 37, 75, 32, 6, Found last non-winning board.
Found last winning board.

Board score: 439
Bingo number: 6
Final score: 2634
